## Additive Secret Sharing

Allows multiple individuals to add numbers together without any person learning anyone else's inputs to the addition.

In [ ]:
import random

x = 5 # number to split

Q = 9872652987365 # Really large Q
n_shares = 3 # split number into k shares
shares = [] # To store the splits

# First two shares in the list are random numbers
for i in range(n_shares-1):
    shares.append(random.randint(0, Q))
# Third share such that it sums to x
final_share = Q - sum(shares) % Q + x
shares.append(final_share)

sum(shares)%Q

5

In [ ]:
# Throwing it to a function
def encrypt(x, n_shares=3, verbose=False):
    shares = []
    for i in range(n_shares-1):
        shares.append(random.randint(0, Q))
    final_share = Q - sum(shares)%Q + x
    shares.append(final_share)
    if verbose==True: print(sum(shares)%Q)
    return tuple(shares)

In [ ]:
encrypt(10, n_shares=15, verbose=True)

10


(7997700394881,
 3562513480914,
 3655037836210,
 7839785859899,
 2424557670628,
 6106404088390,
 5783799953311,
 1006785057742,
 9223253635601,
 6727270744571,
 9832350217298,
 1468717463042,
 4165352678388,
 5151254123512,
 4036440694543)

In [ ]:
# Decryption for global Q
def decrypt(shares):
    return sum(shares)%Q

In [ ]:
decrypt(encrypt(10))

10

In [ ]:
def add(a, b):
    c = []
    assert len(a) == len(b)
    for i in range(len(a)):
        c.append((a[i] + b[i])%Q)
    return tuple(c)

In [ ]:
decrypt(add(encrypt(10), encrypt(15)))

25

In [ ]:
decrypt(add(encrypt(10), encrypt(-15)))

9872652987360

#Yikes!

## Fixed Precision Encoding


The protocol explored above only works for positive integers. But neural network weights are not integers, but floating points. FPE enables computation over decimal numbers using integers.

In [ ]:
# Choose what precision you want to represent
BASE = 10 # Base 10 encoding (decimal)
PRECISION = 4 # 4 decimal places
Q =14765871659873 # large Q value

def encode(x_dec):
    return int(x_dec * (BASE**PRECISION)) % Q

In [ ]:
encode(0.5)

5000

In [ ]:
encode(-0.5) # This wraps around the other side of Q

14765871654873

In [ ]:
def decode(x_fxp):
    return (x_fxp if x_fxp <= Q/2 else x_fxp-Q) / BASE**PRECISION

In [ ]:
decode(encode(0.5))

0.5

In [ ]:
decode(encode(-0.5))

-0.5

In [ ]:
decode(encode(500))

500.0

In [ ]:
decode(encode(-500))

-500.0

## Secret Sharing and Fixed Precision in PySyft

In [ ]:
!pip install syft==0.2.9 >/dev/null

ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.18.5 which is incompatible.
ERROR: google-colab 1.0.0 has requirement notebook~=5.3.0; python_version >= "3.0", but you'll have notebook 5.7.8 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.22.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement tornado~=5.1.0; python_version >= "3.0", but you'll have tornado 4.5.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: bokeh 2.1.1 has requirement tornado>=5.1, but you'll have tornado 4.5.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [ ]:
import numpy as np
import torch
import syft

hook = syft.TorchHook(torch)

bob = syft.VirtualWorker(hook, id='bob')
alice = syft.VirtualWorker(hook, id='alice')

secure_worker = syft.VirtualWorker(hook, id='secure_worker')

In [ ]:
x = torch.tensor([1,2,3,4,5]); x

tensor([1, 2, 3, 4, 5])

In [ ]:
x = x.share(bob, alice, secure_worker); x

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:62478239223 -> bob:73040141759]
	-> [PointerTensor | me:80486766244 -> alice:94447164296]
	-> [PointerTensor | me:42747768110 -> secure_worker:96783813801]
	*crypto provider: me*

We generated 3 different pointers to three shares shared between the workers.

In [ ]:
def objects_print():
    print('Bob: ', bob._objects)
    print('Alice: ', alice._objects)
    print('SecureWorker: ', secure_worker._objects)
objects_print()

Bob:  {20166380940: tensor([ 3587196180313237651, -6888873059417236999, -4360073461282305414,
        -1515237327847543878,   -71044789171378235]), 73040141759: tensor([-6529811432613973911, -5138659850207610264,  2122285321412857269,
        -4788406778992144048, -8173796205866553340])}
Alice:  {63444315841: tensor([ 4738800853575144638,  3279278564924088860,  4827134320224883874,
        -7108230005094848273, -7698097802329685509]), 94447164296: tensor([ 7122423247071752384, -1527703866933724111,  4282105396199056239,
         2786973030881133389, -7439053338164002626])}
SecureWorker:  {95520680817: tensor([-8325997033888382288,  3609594494493148141,  -467060858942578457,
         8623467332942392155,  7769142591501063749]), 96783813801: tensor([ -592611814457778472,  6666363717141334377, -6404390717611913505,
         2001433748111010663, -2833894529678995645])}


In [ ]:
y = x+x

In [ ]:
objects_print()

Bob:  {20166380940: tensor([ 3587196180313237651, -6888873059417236999, -4360073461282305414,
        -1515237327847543878,   -71044789171378235]), 73040141759: tensor([-6529811432613973911, -5138659850207610264,  2122285321412857269,
        -4788406778992144048, -8173796205866553340]), 29184213446: tensor([5387121208481603794, 8169424373294331088, 4244570642825714538,
        8869930515725263520, 2099151661976444936])}
Alice:  {63444315841: tensor([ 4738800853575144638,  3279278564924088860,  4827134320224883874,
        -7108230005094848273, -7698097802329685509]), 94447164296: tensor([ 7122423247071752384, -1527703866933724111,  4282105396199056239,
         2786973030881133389, -7439053338164002626]), 39845071831: tensor([-4201897579566046848, -3055407733867448222,  8564210792398112478,
         5573946061762266778,  3568637397381546364])}
SecureWorker:  {95520680817: tensor([-8325997033888382288,  3609594494493148141,  -467060858942578457,
         8623467332942392155,  776914259

We got one more tensor of 5 elements that depicts the sum x+x, but shared between all three workers. The result is also another additively shared tensor.


In [ ]:
y.get() # You can get the result y.

tensor([ 2,  4,  6,  8, 10])

In [ ]:
objects_print() # it got removed

Bob:  {20166380940: tensor([ 3587196180313237651, -6888873059417236999, -4360073461282305414,
        -1515237327847543878,   -71044789171378235]), 73040141759: tensor([-6529811432613973911, -5138659850207610264,  2122285321412857269,
        -4788406778992144048, -8173796205866553340])}
Alice:  {63444315841: tensor([ 4738800853575144638,  3279278564924088860,  4827134320224883874,
        -7108230005094848273, -7698097802329685509]), 94447164296: tensor([ 7122423247071752384, -1527703866933724111,  4282105396199056239,
         2786973030881133389, -7439053338164002626])}
SecureWorker:  {95520680817: tensor([-8325997033888382288,  3609594494493148141,  -467060858942578457,
         8623467332942392155,  7769142591501063749]), 96783813801: tensor([ -592611814457778472,  6666363717141334377, -6404390717611913505,
         2001433748111010663, -2833894529678995645])}


### What about floats?

In [ ]:
x = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5]); x

tensor([0.1000, 0.2000, 0.3000, 0.4000, 0.5000])

In [ ]:
x = x.fix_prec(); x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300, 400, 500])

This method tensor_object.fix_prec() converted the float tensor to fixed precision tensors under the hood. Although internally, it gets interpreted as floats.

In [ ]:
x = x.float_prec(); x

tensor([0.1000, 0.2000, 0.3000, 0.4000, 0.5000])

In [ ]:
x = x.fix_prec()
type(x)

torch.Tensor

In [ ]:
type(x.child)

syft.frameworks.torch.tensors.interpreters.precision.FixedPrecisionTensor

In [ ]:
x.child

FixedPrecisionTensor>tensor([100, 200, 300, 400, 500])

In [ ]:
x.child.child

tensor([100, 200, 300, 400, 500])

In [ ]:
type(x.child.child)

torch.Tensor

In [ ]:
x = torch.tensor([0.1,0.2, 0.3, 0.4, 0.5]).fix_prec().share(bob, alice, secure_worker); x

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:1899445207 -> bob:57716517966]
	-> [PointerTensor | me:85024294619 -> alice:63610236609]
	-> [PointerTensor | me:15059018317 -> secure_worker:28205769224]
	*crypto provider: me*

In [ ]:
y = x+x; y

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:91435296072 -> bob:95654692142]
	-> [PointerTensor | me:59366224392 -> alice:68426512773]
	-> [PointerTensor | me:45653282888 -> secure_worker:62918500890]
	*crypto provider: me*

In [ ]:
y = y.get().float_prec(); y

In [ ]:
y

tensor([0.2000, 0.4000, 0.6000, 0.8000, 1.0000])

This is the result of the computation